In [3]:
import numpy as np
import csv
import pandas as pd
import time
import matplotlib.pyplot as plt

In [4]:
# Predict via the user-specific median.
# If the user has no data, use the global median.

train_file = 'train.csv'
test_file  = 'test.csv'
soln_file  = 'user_median.csv'
prof_file = 'profiles.csv'
art_file = 'artists.csv'

In [5]:
# Load the training data.
train_data = {}
with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    for row in train_csv:
        user   = row[0]
        artist = row[1]
        plays  = row[2]
    
        if not user in train_data:
            train_data[user] = {}
        
        train_data[user][artist] = int(plays)

In [7]:
# Load the user profile data into dictionaries
prof_data = {}
prof_sex = {}
prof_age = {}
prof_loc = {}
with open(prof_file, 'r') as prof_fh:
    prof_csv = csv.reader(prof_fh, delimiter=',', quotechar='"')
    next(prof_csv, None)
    for row in prof_csv:
        user   = row[0]
        sex = row[1]
        age = row[2]
        loc = row[3]
        # add other
    
        if not user in prof_data:
            prof_data[user] = {}
        
        #train_data[user][artist] = int(plays)
        prof_data[user] = [sex, age, loc]
        # alternatively: store in separate dictionaries?
        prof_sex[user] = sex
        prof_age[user] = age
        prof_loc[user] = loc

In [5]:
df_train = pd.read_csv(train_file)
df_test= pd.read_csv(test_file) # etc if want to handle as pandas df's

In [18]:
df_prof = pd.read_csv(prof_file)
prof_vals = df_prof.values
all_ids = prof_vals.T[:][0]
print all_ids.shape
print all_ids[0]


(233286L,)
fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d


In [19]:
df_arts = pd.read_csv(art_file)
art_vals = df_arts.values
all_arts = art_vals.T[:][0]
print all_arts.shape
print all_arts[0]

(2000L,)
03098741-08b3-4dd7-b3f6-1b0bfa2c879c


In [28]:
# now: PCA analysis
# want to construct matrix where row is one user, columns are artists, entries are listeners
# so, matrix will be 233286 x 2000

s1 = all_ids.shape[0]
s2 = all_arts.shape[0]

data_mat = np.ndarray([s1, s2])
print data_mat.shape

(233286L, 2000L)


In [35]:
# instead: populate using dictionaries -> should be much quicker? no lookups

t = time.time()
# data_mat = np.ndarray([s1, s2]) # this was defined above
# for each user-artist combo
for i in range(s1):
    for j in range(s2):
        usr = all_ids[i]
        art = all_arts[j]
        #try-catch in case no data
        try:
            tmp = train_data[usr][art]
            #print i,',',j
            #print tmp
        except KeyError: # if no corresponding key
            tmp = 0
        # populate matrix
        data_mat[i][j]=tmp
print 'elapsed time: ', t - time.time()
print 'All done!'
        

elapsed time:  -3007.62899995
All done!


In [ ]:
print all_ids[0]
print all_arts[0]

#print train_data[all_ids[0]]
art = 'cd8c5019-5d75-4d5c-bc28-e1e26a7dd5c8'
# find artist id index
tmp = np.equal(art,all_arts)
j = np.nonzero(tmp)[0][0]
print j
art2 = all_arts[j]
print art2
try:
    print train_data[all_ids[0]][all_arts[0]]
except KeyError:
    print 0


In [ ]:
# define script for file writing
def write_to_file(filename, data):
    with open(filename, "w") as f:
        #f.write("Id,Prediction\n")
        for i in range(s1): # for each row
            for j in range(s2): # for each column
                f.write(str(data[i][j]) + ",") # write matrix entry
            f.write("\n") # new line after each row

In [37]:
# output matrix as a CSV file
write_to_file("usr-art_mat.csv", data_mat)
#data_mat.tofile('usr-art_mat.csv',sep=',')

In [53]:
print np.sum(data_mat)

0.0


In [38]:
# find covariance *in* MATLAB?
#print data_mat.shape
Sigma = np.cov(data_mat.T)
print Sigma.shape # should be 2k x 2k

(2000L, 2000L)


In [39]:
# find principal components
L, V = np.linalg.eig(Sigma) # L holds eigenvalues, V holds eigenvectors

In [1]:
# project onto PCA axes
data_proj = np.dot(data_mat,V)

NameError: name 'np' is not defined

In [47]:
print V[:]
#plt.plot(range(2000),)
#plt.show()

[[ -9.53363010e-06  -2.38119052e-05   1.23104825e-05 ...,  -5.33245400e-04
    1.26778732e-03   7.81296072e-04]
 [  3.53203650e-05  -4.87148011e-05   1.12781260e-05 ...,   5.51365545e-04
   -5.94670431e-04  -1.95761484e-04]
 [  3.02338075e-05  -2.36689274e-05   8.85349186e-06 ...,  -3.18407654e-02
   -6.32800472e-02  -3.77361364e-02]
 ..., 
 [  4.09129315e-06   7.07845112e-06   7.46890654e-06 ...,   1.60515820e-02
    8.67354550e-04  -5.54315141e-03]
 [ -1.15388447e-06  -7.22470524e-06   5.95583753e-06 ...,  -2.42958948e-04
   -2.98450678e-02   4.67839829e-03]
 [ -1.54816944e-05  -4.80399776e-05   2.37139089e-05 ...,   2.30401806e-03
   -5.25807754e-04   7.29334249e-04]]


In [ ]:
# alternatively? subtract mean first to get more compressed PC's
# PC's will then capture changes relative to some average profile

In [ ]:
# Compute the global median and per-user median.
plays_array  = []
user_medians = {}
for user, user_data in train_data.iteritems():
    user_plays = []
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
        user_plays.append(plays)

    user_medians[user] = np.median(np.array(user_plays))
global_median = np.median(np.array(plays_array))

In [ ]:
# Write out test solutions.
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            user   = row[1]
            artist = row[2]

            if user in user_medians:
                soln_csv.writerow([id, user_medians[user]])
            else:
                print "User", id, "not in training data."
                soln_csv.writerow([id, global_median])
                
